In [2]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00


In [4]:
sample_rate = 16000

# loading dataset

In [6]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import Wav2Vec2FeatureExtractor

# Load your dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy")

# Get the first entry from the validation dataset
first_entry = ds['validation'][0]

# Access the audio array and sampling rate
audio_array = first_entry['audio']['array']
sampling_rate = first_entry['audio']['sampling_rate']

# prediction on individual audio sample

emotions = ['anger', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [12]:
import os
import random
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import Audio
from datasets import load_dataset
from transformers import AutoConfig, Wav2Vec2FeatureExtractor, HubertPreTrainedModel, HubertModel

# Define model parameters
model_name_or_path = "xmj2002/hubert-base-ch-speech-emotion-recognition"
duration = 6
sample_rate = 16000

# Load the configuration for the model
config = AutoConfig.from_pretrained(model_name_or_path)

# Emotion classification function
def id2class(id):
    if id == 0:
        return "angry"
    elif id == 1:
        return "fear"
    elif id == 2:
        return "happy"
    elif id == 3:
        return "neutral"
    elif id == 4:
        return "sad"
    else:
        return "surprise"

# Define the prediction function
# def predict(audio_array, processor, model):
#     speech = processor(audio_array, padding="max_length", truncation=True, max_length=duration * sample_rate,
#                        return_tensors="pt", sampling_rate=sample_rate).input_values
#     with torch.no_grad():
#         logit = model(speech)
#     score = F.softmax(logit, dim=1).detach().cpu().numpy()[0]
#     emotion_id = torch.argmax(logit).cpu().numpy()
#     emotion_class = id2class(emotion_id)
#     print(f"Predicted Emotion: {emotion_class} \t Confidence: {score[emotion_id]:.4f}")

def predict(audio_array, processor, model):
    # Prepare the input array for the model
    speech = processor(audio_array, padding="max_length", truncation=True, max_length=duration * sample_rate,
                       return_tensors="pt", sampling_rate=sample_rate).input_values

    with torch.no_grad():
        logit = model(speech)

    # Get the softmax scores
    score = F.softmax(logit, dim=1).detach().cpu().numpy()[0]
    emotion_id = torch.argmax(logit).cpu().numpy()

    # Convert ID to emotion class
    emotion_class = id2class(emotion_id)

    # Return both the predicted emotion and confidence score
    return emotion_class, score[emotion_id]


# Define the Hubert Classification Head
class HubertClassificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_class)

    def forward(self, x):
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

# Define the Hubert model for speech classification
class HubertForSpeechClassification(HubertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.hubert = HubertModel(config)
        self.classifier = HubertClassificationHead(config)
        self.init_weights()

    def forward(self, x):
        outputs = self.hubert(x)
        hidden_states = outputs[0]
        x = torch.mean(hidden_states, dim=1)
        x = self.classifier(x)
        return x

# # Load the dataset
# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy")

# # Get the first entry from the validation dataset
# first_entry = ds['validation'][0]

# # Access the audio array and sampling rate
# audio_array = first_entry['audio']['array']
# sampling_rate = first_entry['audio']['sampling_rate']

# Resample the audio array if necessary
if sampling_rate != sample_rate:
    print(f"Resampling from {sampling_rate}Hz to {sample_rate}Hz")
    audio_array = librosa.resample(audio_array, orig_sr=sampling_rate, target_sr=sample_rate)

# Initialize the processor and model
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
model = HubertForSpeechClassification.from_pretrained(model_name_or_path)
model.eval()

# Send the audio array to the model for prediction
# predict(audio_array, processor, model)


Some weights of the model checkpoint at xmj2002/hubert-base-ch-speech-emotion-recognition were not used when initializing HubertForSpeechClassification: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSpeechClassification were not initialized from the model checkpoint at xmj2002/hubert-base-ch-speech-emotion-recognition and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encod

HubertForSpeechClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder

In [13]:
output = predict(audio_array, processor, model)
output

('fear', 0.9730136)

In [14]:
type(output)

tuple

# predicting for whole dataset

In [16]:
predictions = []


for entry in ds['validation']:
    audio_array = entry['audio']['array']
    sampling_rate = entry['audio']['sampling_rate']

    # Resample if necessary
    if sampling_rate != sample_rate:
        audio_array = librosa.resample(audio_array, orig_sr=sampling_rate, target_sr=sample_rate)

    try:
        # Get prediction
        output = predict(audio_array, processor, model)

        if output is not None:  # Check if output is valid
            emotion_class, confidence = output
            print(f"{emotion_class}, {confidence}")

            # Append to predictions list
            predictions.append({"emotion": emotion_class, "confidence": confidence})

    except Exception as e:
        # Use a different key that exists in the entry if 'uri' doesn't
        entry_info = entry.get('audio', {}).get('path', 'unknown')
        print(f"Error processing entry: {entry_info}, Error: {e}")

# Create a DataFrame from the predictions list
df_ser = pd.DataFrame(predictions)

# Display the DataFrame with predictions
df_ser

fear, 0.9730135798454285
fear, 0.4743833541870117
fear, 0.702656626701355
fear, 0.8861019015312195
surprise, 0.8362240791320801
fear, 0.5270298719406128
fear, 0.6904441118240356
sad, 0.49222058057785034
sad, 0.9920583367347717
fear, 0.503082811832428
surprise, 0.5603905916213989
fear, 0.5386357307434082
surprise, 0.8774287700653076
fear, 0.6674942374229431
sad, 0.9986357092857361
fear, 0.697052538394928
fear, 0.7067537307739258
sad, 0.9792997241020203
sad, 0.9673706889152527
surprise, 0.9881681799888611
sad, 0.9964746832847595
sad, 0.5509658455848694
sad, 0.7437800765037537
sad, 0.6652547121047974
surprise, 0.8896306157112122
fear, 0.7642313241958618
surprise, 0.7463772296905518
surprise, 0.5512852072715759
sad, 0.998322069644928
surprise, 0.9762346744537354
sad, 0.9810918569564819
sad, 0.9991068243980408
fear, 0.8330680727958679
sad, 0.9998229146003723
sad, 0.984714686870575
sad, 0.8823534250259399
sad, 0.9634436964988708
sad, 0.9276549220085144
sad, 0.9796915650367737
sad, 0.90970373

,emotion,confidence
0,fear,0.973014
1,fear,0.474383
2,fear,0.702657
3,fear,0.886102
4,surprise,0.836224
...,...,...
68,sad,0.990934
69,fear,0.731559
70,fear,0.706702
71,fear,0.534012


In [17]:
df_ser.to_csv("ser_output.csv")